In [1]:
from scipy.optimize import linprog

c = [-1, -2]   # maximize x1 + 2x2 -> minimize -x1 - 2x2
A = [[2, 1],
     [1, 2]]
b = [20, 20]
bounds = [(0, None), (0, None)]

res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method="highs")
print(res)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -20.0
              x: [ 0.000e+00  1.000e+01]
            nit: 1
          lower:  residual: [ 0.000e+00  1.000e+01]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 1.000e+01  0.000e+00]
                 marginals: [-0.000e+00 -1.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0


In [2]:
import numpy as np
from scipy.optimize import minimize

def f(z):
    x, y = z
    return (1 - x)**2 + 100*(y - x**2)**2

x0 = np.array([-1.2, 1.0])  # initial guess
res = minimize(f, x0, method="BFGS", options={"gtol": 1e-8, "maxiter": 1000})
print("Success:", res.success, "-", res.message)
print("x*:", res.x, " f(x*):", res.fun)

Success: True - Optimization terminated successfully.
x*: [0.99999552 0.99999104]  f(x*): 2.0056016706663067e-11


In [5]:
from scipy.optimize import NonlinearConstraint

def g(v):
    x, y = v
    return x*y - 9

nl_con = NonlinearConstraint(g, -np.inf, 0.0)

res = minimize(f, x0, method="trust-constr",
               constraints=[nl_con], bounds=bounds)

# --- Print solution info ---
print("Success:", res.success)
print("Message:", res.message)
print("Status code:", res.status)
print("Objective value f(x*):", res.fun)
print("Optimal x:", res.x)
print("Number of iterations:", res.nit)
print("Number of function evaluations:", res.nfev)
print("Number of gradient evaluations:", res.njev)
print("Number of Hessian evaluations:", res.nhev)


Success: True
Message: `xtol` termination condition is satisfied.
Status code: 2
Objective value f(x*): 1.482114827940386e-14
Optimal x: [0.99999991 0.99999983]
Number of iterations: 127
Number of function evaluations: 585
Number of gradient evaluations: 195
Number of Hessian evaluations: 0


In [6]:
from math import cos
from scipy.optimize import brentq

f = lambda x: cos(x) - x
root = brentq(f, 0, 1)  # f(0)>0, f(1)<0  → bracketed
print("root:", root)

root: 0.7390851332151559


In [7]:
import cvxpy as cp

x = cp.Variable()
y = cp.Variable()

objective = cp.Maximize(x + 2*y)
constraints = [2*x + y <= 20,
               x + 2*y <= 20,
               x >= 0, y >= 0]

prob = cp.Problem(objective, constraints)
result = prob.solve(solver=cp.GUROBI)
print("Optimal value:", result)
print("x =", x.value, "y =", y.value)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-14
Optimal value: 20.0
x = 0.0 y = 10.0


In [8]:
from pyomo.environ import *

model = ConcreteModel()
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

model.obj = Objective(expr=model.x + 2*model.y, sense=maximize)
model.con1 = Constraint(expr=2*model.x + model.y <= 20)
model.con2 = Constraint(expr=model.x + 2*model.y <= 20)

solver = SolverFactory("gurobi")
solver.solve(model)

print("x =", model.x(), "y =", model.y())

x = 0.0 y = 10.0


In [9]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("example")
x = m.addVar(lb=0, name="x")
y = m.addVar(lb=0, name="y")

m.setObjective(x + 2*y, GRB.MAXIMIZE)
m.addConstr(2*x + y <= 20, "c1")
m.addConstr(x + 2*y <= 20, "c2")

m.optimize()
for v in m.getVars():
    print(f"{v.VarName} = {v.X}")
print("Objective value =", m.ObjVal)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 22.6.0 22G120)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xe9c063e4
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+30   3.000000e+30   3.000000e+00      0s
       2    2.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.000000000e+01
x = 0.0
y = 10.0
Objective value = 20.0


In [10]:
m2 = gp.Model("mip_demo")
x = m2.addVar(vtype=GRB.BINARY, name="x")
y = m2.addVar(vtype=GRB.INTEGER, lb=0, name="y")

m2.setObjective(3*x + 2*y, GRB.MAXIMIZE)
m2.addConstr(2*x + y <= 4, "cap")
m2.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 22.6.0 22G120)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xe21ddf3a
Variable types: 0 continuous, 2 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
Found heuristic solution: objective 7.0000000
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 2: 8 7 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e+00, best bound 8.000000000000e+00, gap 0.0000%


In [11]:
m = gp.Model("qcp_demo")

x = m.addVar(lb=-GRB.INFINITY, name="x")
y = m.addVar(lb=-GRB.INFINITY, name="y")

# Objective: minimize x + y
m.setObjective(x + y, GRB.MINIMIZE)

# Quadratic constraint: x^2 + y^2 <= 1  (unit circle)
m.addQConstr(x*x + y*y <= 1, "qc1")

m.optimize()
print("x =", x.X, " y =", y.X, " obj =", m.ObjVal)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 22.6.0 22G120)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x4b2adfa8
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 1 rows, 3 columns, 1 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   0.00000000e+00  0.00000000e+00  0.00e+00 3.16e+

In [12]:
from docplex.mp.model import Model

mdl = Model(name="example")

# Variables
x = mdl.continuous_var(name="x", lb=0)
y = mdl.continuous_var(name="y", lb=0)

# Objective: maximize x + 2y
mdl.maximize(x + 2*y)

# Constraints
mdl.add_constraint(2*x + y <= 20, "c1")
mdl.add_constraint(x + 2*y <= 20, "c2")

# Solve
solution = mdl.solve(log_output=True)
print("x =", x.solution_value, "y =", y.solution_value)
print("Objective value =", mdl.objective_value)

Version identifier: 22.1.0.0 | 2022-03-27 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =            20.000000
x = 0 y = 10.0
Objective value = 20.0


In [14]:
from docplex.mp.model import Model

mdl = Model('mip_demo')
x = mdl.binary_var(name='x')                   # 0/1
y = mdl.integer_var(lb=0, name='y')           # integer ≥ 0

mdl.maximize(3*x + 2*y)
mdl.add_constraint(2*x + y <= 4, 'cap')

sol = mdl.solve(log_output=True)
print("x =", x.solution_value, " y =", y.solution_value, " obj =", mdl.objective_value)

Version identifier: 22.1.0.0 | 2022-03-27 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 1 rows and 2 columns.
MIP Presolve added 1 rows and 1 columns.
Aggregator did 1 substitutions.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.00 ticks)
Parallel b&c, 10 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.00 ticks)
x = 0  y = 4.0  obj = 8.0


In [15]:
m.write("./example_model.mps")

In [16]:
import gurobipy as gp
m = gp.read("example_model.mps")
m.optimize()

Read MPS format model from file example_model.mps
Reading time = 0.01 seconds
qcp_demo: 0 rows, 2 columns, 0 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 22.6.0 22G120)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x4b2adfa8
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 1 rows, 3 columns, 1 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter      

In [17]:
from docplex.mp.model_reader import ModelReader

mr = ModelReader()                     # uses local CPLEX if present
mdl = mr.read_model('example_model.mps')
mdl.solve(log_output=True)

Version identifier: 22.1.0.0 | 2022-03-27 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced QCP has 1 rows, 3 columns, and 1 nonzeros.
Reduced QCP has 1 quadratic constraints.
Presolve time = 0.00 sec. (0.00 ticks)
Parallel mode: using up to 10 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 0
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.00 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 10
  Rows in Factor            = 1
  Integer space required    = 1
  Total non-zeros in factor = 1
  Total FP ops to factor    = 1
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf Inf Ratio
   0   0.0000000e+00   0.0000000e+00  1.41e+00  0.00e+00  4.41e+00  1.00e+00
   1  -1.4334743e+00  -1.3112928e+00  1.41e+00  0.00e+00  4.41e+00  5.82e+00
   2  -1.4142198e+00  -1.4138933e+00  7.03e-02  0.00e+00  2.19e-01  1.93e+03
   3  -1.4142136e+00  -1.4142

docplex.mp.solution.SolveSolution(obj=-1.41421,values={x:-0.707107,y:-0...